In [2]:
# -*- coding: utf-8 -*-
from __future__ import print_function
import os
import collections

import nltk
import numpy as np
from keras.callbacks import TensorBoard
from keras.layers import Dense, Dropout, Conv1D, Embedding, GlobalMaxPooling1D
from keras.models import Sequential
from keras.preprocessing.sequence import pad_sequences
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
import codecs

np.random.seed(42)

Using TensorFlow backend.


In [1]:
!mkdir data
!wget https://raw.githubusercontent.com/chen0040/keras-sentiment-analysis-web-api/master/demo/data/umich-sentiment-train.txt -P ./data

mkdir: cannot create directory ‘data’: File exists
--2018-11-05 05:31:38--  https://raw.githubusercontent.com/chen0040/keras-sentiment-analysis-web-api/master/demo/data/umich-sentiment-train.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 447539 (437K) [text/plain]
Saving to: ‘./data/umich-sentiment-train.txt’

umich-sentiment-tra 100%[===================>] 437.05K  --.-KB/s    in 0.08s   

2018-11-05 05:31:38 (5.14 MB/s) - ‘./data/umich-sentiment-train.txt’ saved [447539/447539]



In [3]:
INPUT_FILE = "./data/umich-sentiment-train.txt"
LOG_DIR = "./logs"
VOCAB_SIZE = 5000
EMBED_SIZE = 100
NUM_FILTERS = 256
NUM_WORDS = 3
BATCH_SIZE = 64
NUM_EPOCHS = 20

counter = collections.Counter()
with codecs.open(INPUT_FILE, "r", encoding="utf-8") as fin:
    maxlen = 0
    for line in fin:
        _, sent = line.strip().split("\t")
        try:
            words = [x.lower() for x in nltk.word_tokenize(sent)]
        except LookupError:
            print("Englisth tokenize does not downloaded. So download it.")
            nltk.download("punkt")
            words = [x.lower() for x in nltk.word_tokenize(sent)]
        maxlen = max(maxlen, len(words))
        for word in words:
            counter[word] += 1

word2index = collections.defaultdict(int)
for wid, word in enumerate(counter.most_common(VOCAB_SIZE)):
    word2index[word[0]] = wid + 1
vocab_sz = len(word2index) + 1
index2word = {v: k for k, v in word2index.items()}

xs, ys = [], []
with codecs.open(INPUT_FILE, "r", encoding="utf-8") as fin:
    for line in fin:
        label, sent = line.strip().split("\t")
        ys.append(int(label))
        words = [x.lower() for x in nltk.word_tokenize(sent)]
        wids = [word2index[word] for word in words]
        xs.append(wids)

X = pad_sequences(xs, maxlen=maxlen)
Y = np_utils.to_categorical(ys)

Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size=0.3, 
                                                random_state=42)
print(Xtrain.shape, Xtest.shape, Ytrain.shape, Ytest.shape)

Englisth tokenize does not downloaded. So download it.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
(4960, 42) (2126, 42) (4960, 2) (2126, 2)


In [4]:
model = Sequential()
model.add(Embedding(vocab_sz, EMBED_SIZE, input_length=maxlen))
model.add(Dropout(0.2))
model.add(Conv1D(filters=NUM_FILTERS, kernel_size=NUM_WORDS, activation="relu"))
model.add(GlobalMaxPooling1D())
model.add(Dense(2, activation="softmax"))
model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 42, 100)           232900    
_________________________________________________________________
dropout_1 (Dropout)          (None, 42, 100)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 40, 256)           77056     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 514       
Total params: 310,470
Trainable params: 310,470
Non-trainable params: 0
_________________________________________________________________


In [5]:
model.compile(optimizer="adam", loss="categorical_crossentropy",
              metrics=["accuracy"])

history = model.fit(Xtrain, Ytrain, batch_size=BATCH_SIZE,
                    epochs=NUM_EPOCHS,
                    callbacks=[TensorBoard(LOG_DIR)],
                    validation_data=(Xtest, Ytest))

# evaluate model
score = model.evaluate(Xtest, Ytest, verbose=1)
print("Test score: {:.3f}, accuracy: {:.3f}".format(score[0], score[1]))

Train on 4960 samples, validate on 2126 samples
Epoch 1/20
4960/4960 [==============================] - 4s 782us/step - loss: 0.3100 - acc: 0.8911 - val_loss: 0.0395 - val_acc: 0.9845
Epoch 2/20
4960/4960 [==============================] - 1s 139us/step - loss: 0.0205 - acc: 0.9923 - val_loss: 0.0208 - val_acc: 0.9925
Epoch 3/20
4960/4960 [==============================] - 1s 155us/step - loss: 0.0071 - acc: 0.9980 - val_loss: 0.0175 - val_acc: 0.9939
Epoch 4/20
4960/4960 [==============================] - 1s 156us/step - loss: 0.0030 - acc: 0.9994 - val_loss: 0.0167 - val_acc: 0.9953
Epoch 5/20
4960/4960 [==============================] - 1s 157us/step - loss: 0.0019 - acc: 0.9996 - val_loss: 0.0164 - val_acc: 0.9934
Epoch 6/20
4960/4960 [==============================] - 1s 164us/step - loss: 0.0012 - acc: 0.9998 - val_loss: 0.0160 - val_acc: 0.9929
Epoch 7/20
4960/4960 [==============================] - 1s 160us/step - loss: 0.0013 - acc: 0.9998 - val_loss: 0.0161 - val_acc: 0.9958
